In [2]:
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu113 (from versions: 1.9.0, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.11.0+cu113
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__

TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda

CUDA = format_cuda_version(CUDA_version)
print(TORCH, CUDA)

AttributeError: 'NoneType' object has no attribute 'replace'

In [2]:
import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')

import os
import numpy as np
import pandas as pd
import json,pickle
import networkx as nx
from math import sqrt
from random import shuffle
from collections import OrderedDict
from scipy import stats
from IPython.display import SVG
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import MolFromSmiles
from torch_geometric import data as DATA
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_max_pool as gmp
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

# 시각화 라이브러리
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

(예제)smile를 이용해서 분자 그래프 형식으로 변환해보기

In [11]:
def moltosvg(mol,molSize=(600,400),kekulize=True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return svg

In [9]:
def render_svg(svg):
    return SVG(svg.replace('svg:',''))

In [ ]:
example_smile = 'CC(=O)Nc1ccc(cc1)O'
example_mol = Chem.MolFromSmiles(example_smile)
render_svg(moltosvg(example_mol))

Atom을 one hot encoding으로 바꾸는 함수들

In [14]:
def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

In [15]:
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

In [16]:
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    [atom.GetIsAromatic()])

[2] 실제 SMILES를 이용해서 Graph 결과 만들기
SMILES의 Return: 원자 개수, 원자 특성 행렬, 인접 행렬

In [18]:
def smile_to_graph(smile):
    # SMILES 문자열로부터 분자 그래프 데이터 생성
    mol = Chem.MolFromSmiles(smile)
    
    # mol.GetNumAtoms() : 분자에 소속되어 있는 원자의 개수
    c_size = mol.GetNumAtoms()
    
    features = []

    # 분자에 소속되어 있는 원자들을 순회하면서 원자 특성 정보 수집
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    # 분자를 이루는 원자들의 연결 구조 정보를 순회하면서 인접 정보 수집
    # 연결 구조 정보 : 시작 원자 index, 끝 원자 index
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])

    # 연결 구조 정보를 통한 방향 그래프 생성    
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])
  
    return c_size, features, edge_index

smiles to graph 적용 함수를 이해하기 위해 쪼개서 결과보는 것들임

In [19]:
#구성하는 원자의 개수 
example_c_size = example_mol.GetNumAtoms()
example_c_size

11

In [20]:
#feature 행렬 출력 
example_features = []

for atom in example_mol.GetAtoms():
    feature = atom_features(atom)
    example_features.append(feature/sum(feature))

example_features[0]

array([0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  ])

In [21]:
#원자들의 결합 정보
example_edges = []

for bond in example_mol.GetBonds():
    example_edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    
example_edges

[[0, 1],
 [1, 2],
 [1, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [7, 10],
 [9, 4]]

In [22]:
#networkx library 활용해 방향 그래프로 데이터 변환
example_graph = nx.Graph(example_edges).to_directed()

#index 정보추가
example_edge_index = [] 

for e1, e2 in example_graph.edges:
    example_edge_index.append([e1,e2])

#방향 그래프를 인접 행렬로 변환 
adj_matrix = nx.adjacency_matrix(example_graph)
print(adj_matrix.todense())

[[0 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]]


[2] Protein Sequence

In [24]:

seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"

seq_dict = {v:(i+1) for i,v in enumerate(seq_voc)} #Q 원래 이런식으로 dictionary를 만드는 건가? 그냥 예시용일까?

seq_dict_len = len(seq_dict)

#Padding max sequence
max_seq_len = 1000


In [25]:
def seq_cat(prot):
    x = np.zeros(max_seq_len) #크기가 1000이고 원소가 0인 배열
    
    for i, ch in enumerate(prot[:max_seq_len]): #sequence dictionary
        x[i] = seq_dict[ch]
    return x
    

In [30]:
#[Uneed] 예시 protein sequence
example_csv = pd.read_csv('davis_train.csv')
example_prot_seq = example_csv.target_sequence[7]
print('7th PROT seq :', example_prot_seq)
print('Num of protein sequence : ', len(example_prot_seq))

example_seq_cat = seq_cat(example_prot_seq)

7th PROT seq : MELENIVANSLLLKARQGGYGKKSGRSKKWKEILTLPPVSQCSELRHSIEKDYSSLCDKQPIGRRLFRQFCDTKPTLKRHIEFLDAVAEYEVADDEDRSDCGLSILDRFFNDKLAAPLPEIPPDVVTECRLGLKEENPSKKAFEECTRVAHNYLRGEPFEEYQESSYFSQFLQWKWLERQPVTKNTFRHYRVLGKGGFGEVCACQVRATGKMYACKKLQKKRIKKRKGEAMALNEKRILEKVQSRFVVSLAYAYETKDALCLVLTIMNGGDLKFHIYNLGNPGFDEQRAVFYAAELCCGLEDLQRERIVYRDLKPENILLDDRGHIRISDLGLATEIPEGQRVRGRVGTVGYMAPEVVNNEKYTFSPDWWGLGCLIYEMIQGHSPFKKYKEKVKWEEVDQRIKNDTEEYSEKFSEDAKSICRMLLTKNPSKRLGCRGEGAAGVKQHPVFKDINFRRLEANMLEPPFCPDPHAVYCKDVLDIEQFSVVKGIYLDTADEDFYARFATGCVSIPWQNEMIESGCFKDINKSESEEALPLDLDKNIHTPVSRPNRGFFYRLFRRGGCLTMVPSEKEVEPKQC
Num of protein sequence :  578


실제 코드

In [ ]:
all_prots = [] 
datasets = ['davis','kiba']

kiba_test = pd.read_csv('kiba_test.csv')
print(kiba_test)


In [ ]:
for dataset in datasets:
    
    fpath = dataset+'/'
    train_fold = json.load 